<a href="https://colab.research.google.com/github/TFMUCM01/TFM/blob/main/marco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Paso 1: Instalación de librerías
# Se instalarán las librerías necesarias de Hugging Face y Pandas.
!pip install transformers pandas torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [2]:
# Paso 2: Importar librerías
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np # Para manejar las probabilidades

In [3]:
# Paso 3: Cargar el modelo y el tokenizer pre-entrenados
# Define el nombre del modelo que identificamos en Hugging Face.
model_name = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

In [7]:
# Carga el tokenizer asociado al modelo.
# El tokenizer es crucial para convertir el texto en un formato que el modelo pueda entender.
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
# Carga el modelo pre-entrenado para clasificación de secuencias.
# .to('cuda') intenta mover el modelo a la GPU si está disponible para una inferencia más rápida.
# Si no tienes GPU o prefieres usar CPU, puedes omitir .to('cuda') o usar .to('cpu').
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval() # Pone el modelo en modo de evaluación (desactiva el dropout, etc.)

print(f"Modelo y tokenizer cargados correctamente en {device}.")

Modelo y tokenizer cargados correctamente en cpu.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/UCM/TFM/bloomberg_europe_2024.csv')

Mounted at /content/drive


In [9]:
df.head()

,fecha,titular,url_archivo
0,"20240110,""RUIN: Money, Ego and Deception at FT...",NaN,NaN
1,20240110,Also streaming on your TV:,https://web.archive.org/web/20240111000550/htt...
2,20240110,Ex-Credit Suisse Star Trader Lemssouguer’s Hed...,https://web.archive.org/web/20240111000550/htt...
3,20240110,Meloni Pushes Hungary’s Orban to Unlock Ukrain...,https://web.archive.org/web/20240111000550/htt...
4,20240118,Also streaming on your TV:,https://web.archive.org/web/20240118235629/htt...


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   fecha        210 non-null    object
 1   titular      115 non-null    object
 2   url_archivo  115 non-null    object
dtypes: object(3)
memory usage: 5.1+ KB


In [13]:
duplicados = df[df.duplicated()]
print(duplicados)

                                                 fecha  \
67   20240608,"OpenAI E1, Most Silicon Valley Man A...   
69   20240610,"OpenAI E1, Most Silicon Valley Man A...   
71   20240615,"OpenAI E2, Ilya Dreams of AGI",https...   
73   20240620,"OpenAI E2, Ilya Dreams of AGI",https...   
75   20240621,"OpenAI E3, Heaven and Hell, Part 1",...   
77   20240622,"OpenAI E3, Heaven and Hell, Part 1",...   
79   20240625,"OpenAI E3, Heaven and Hell, Part 1",...   
81   20240626,"OpenAI E3, Heaven and Hell, Part 1",...   
83   20240627,"OpenAI E3, Heaven and Hell, Part 1",...   
85   20240628,"OpenAI E3, Heaven and Hell, Part 1",...   
87   20240701,"Foundering: OpenAI E4, Heaven and He...   
89   20240702,"Foundering: OpenAI E4, Heaven and He...   
91   20240703,"Foundering: OpenAI E4, Heaven and He...   
93   20240709,"Foundering: OpenAI E4, Heaven and He...   
95   20240710,"OpenAI E5, Beware the Ides of Novemb...   
99                                            20241004   
102           

In [15]:
duplicados.count()

,0
fecha,47
titular,29
url_archivo,29


In [10]:
# Paso 5: Definir la función de predicción de sentimiento (VERSIÓN CORREGIDA)
def get_sentiment(text):
    """
    Realiza el análisis de sentimiento para un texto dado utilizando el modelo cargado.
    Devuelve la etiqueta de sentimiento (positivo/negativo/neutral) y las probabilidades.
    """
    if not isinstance(text, str) or pd.isna(text):
        # Asegurarse de que las probabilidades sean NaN si el texto no es válido
        return 'N/A', np.nan, np.nan, np.nan # Ahora devuelve 4 valores: sentimiento, prob_pos, prob_neg, prob_neutral

    # Tokenizar el texto
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()} # Mover inputs al mismo dispositivo que el modelo

    # Realizar la inferencia
    with torch.no_grad():
        outputs = model(**inputs)

    # Obtener las probabilidades
    probabilities = torch.softmax(outputs.logits, dim=1)[0].cpu().numpy()

    # ******* ¡CAMBIO IMPORTANTE AQUÍ! *******
    # Ajustar el mapeo de etiquetas según la configuración común de financial_phrasebank:
    # 0: Negativo, 1: Neutral, 2: Positivo
    label_mapping = {0: 'Negativo', 1: 'Neutral', 2: 'Positivo'}

    # Determinar el sentimiento predicho
    predicted_class_id = np.argmax(probabilities)
    sentiment = label_mapping[predicted_class_id]

    # Obtener las probabilidades individuales según el mapeo
    prob_negative = probabilities[0] # Probabilidad de ser negativo (índice 0)
    prob_neutral = probabilities[1]  # Probabilidad de ser neutral (índice 1)
    prob_positive = probabilities[2] # Probabilidad de ser positivo (índice 2)

    # Devuelve el sentimiento, probabilidad de positivo, probabilidad de negativo y probabilidad de neutral
    return sentiment, prob_positive, prob_negative, prob_neutral

In [11]:
# Paso 6: Aplicar la función de predicción a tu DataFrame
print("\nIniciando el análisis de sentimiento...")

# Crear listas para almacenar los resultados (ahora incluyendo neutral)
sentimientos = []
probabilidades_positivas = []
probabilidades_negativas = []
probabilidades_neutrales = [] # Nueva lista para la probabilidad neutral

# Iterar sobre cada fila del DataFrame y aplicar la función
for index, row in df.iterrows():
    titular = row['titular']
    # La función ahora devuelve 4 valores, así que los capturamos todos
    sentiment, prob_pos, prob_neg, prob_neut = get_sentiment(titular)
    sentimientos.append(sentiment)
    probabilidades_positivas.append(prob_pos)
    probabilidades_negativas.append(prob_neg)
    probabilidades_neutrales.append(prob_neut) # Añadir la probabilidad neutral

# Añadir los resultados como nuevas columnas al DataFrame
df['sentimiento_resultado'] = sentimientos
df['probabilidad_positivo'] = probabilidades_positivas
df['probabilidad_negativa'] = probabilidades_negativas
df['probabilidad_neutral'] = probabilidades_neutrales # Nueva columna

print("\nAnálisis de sentimiento completado.")


Iniciando el análisis de sentimiento...

Análisis de sentimiento completado.


In [12]:
# Paso 7: Mostrar los resultados
print("\nDataFrame con los resultados del análisis de sentimiento:")
print(df[['fecha', 'titular', 'sentimiento_resultado', 'probabilidad_positivo', 'probabilidad_negativa', 'probabilidad_neutral']].head())


DataFrame con los resultados del análisis de sentimiento:
                                               fecha  \
0  20240110,"RUIN: Money, Ego and Deception at FT...   
1                                           20240110   
2                                           20240110   
3                                           20240110   
4                                           20240118   

                                             titular sentimiento_resultado  \
0                                                NaN                   N/A   
1                         Also streaming on your TV:               Neutral   
2  Ex-Credit Suisse Star Trader Lemssouguer’s Hed...               Neutral   
3  Meloni Pushes Hungary’s Orban to Unlock Ukrain...              Positivo   
4                         Also streaming on your TV:               Neutral   

   probabilidad_positivo  probabilidad_negativa  probabilidad_neutral  
0                    NaN                    NaN                

In [13]:
# Paso 8: Guardar los resultados en un nuevo archivo CSV
# Define el nombre del archivo de salida.
output_file_name = 'noticias_financieras_con_sentimiento.csv'

# Guarda el DataFrame a CSV.
# `index=False` es importante para no incluir la columna de índice de pandas en el CSV.
df.to_csv(output_file_name, index=False)

print(f"\n¡Listo! Los resultados han sido guardados en '{output_file_name}' en tu entorno de Colab.")

# Paso 9 (Opcional): Descargar el archivo a tu computadora local
# Ejecuta estas líneas si quieres descargar el CSV a tu dispositivo.
from google.colab import files
files.download(output_file_name)

print(f"El archivo '{output_file_name}' se está descargando a tu máquina local.")


¡Listo! Los resultados han sido guardados en 'noticias_financieras_con_sentimiento.csv' en tu entorno de Colab.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

El archivo 'noticias_financieras_con_sentimiento.csv' se está descargando a tu máquina local.
